In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from category_encoders import TargetEncoder

In [16]:
df=pd.read_csv("../Processed Data/prd_Master.csv")

In [18]:
df['order_purchase_timestamp'] = pd.to_datetime(
    df['order_purchase_timestamp'],
    errors='coerce'
)

df['snapshot_date'] = pd.to_datetime(
    df['snapshot_date'],
    errors='coerce'
)


In [19]:
repeat_df = df[df['is_repeat_customer'] == 1].copy()


In [20]:
repeat_df = repeat_df.sort_values(
    ['customer_unique_id', 'order_purchase_timestamp']
)


In [21]:
repeat_df['prev_order_time'] = (
    repeat_df
    .groupby('customer_unique_id')['order_purchase_timestamp']
    .shift(1)
)

repeat_df['gap_days'] = (
    repeat_df['order_purchase_timestamp'] - repeat_df['prev_order_time']
).dt.days


In [22]:
second_gap = repeat_df.groupby('customer_unique_id')['gap_days'].nth(1)
second_gap = second_gap.dropna()


In [23]:
second_gap.describe(percentiles=[.25,.5,.75,.9,.95])


count    222.000000
mean      59.851351
std       76.270301
min        0.000000
25%        1.250000
50%       37.000000
75%       83.000000
90%      172.300000
95%      224.000000
max      375.000000
Name: gap_days, dtype: float64

In [24]:
for w in [30, 60, 90, 180, 365]:
    print(f"{w} days:", (second_gap <= w).mean())


30 days: 0.47297297297297297
60 days: 0.6711711711711712
90 days: 0.7612612612612613
180 days: 0.9099099099099099
365 days: 0.990990990990991
